<a href="https://colab.research.google.com/github/priya003689/facialEmotions/blob/main/facial_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless numpy gradio deepface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.1/197.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.5 MB/s eta 0:00:00


In [ ]:


import cv2
import numpy as np
import gradio as gr
from deepface import DeepFace


face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)


def detect_emotion_and_draw(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:


        pad = int(0.25 * w)
        x1 = max(x - pad, 0)
        y1 = max(y - pad, 0)
        x2 = min(x + w + pad, img.shape[1])
        y2 = min(y + h + pad, img.shape[0])

        face = img[y1:y2, x1:x2]

        face = cv2.resize(face, (224, 224))

        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        try:
            analysis = DeepFace.analyze(
                face_rgb,
                actions=['emotion'],
                enforce_detection=False
            )


            if isinstance(analysis, list):
                emotion = analysis[0]['dominant_emotion']
            else:
                emotion = analysis['dominant_emotion']

            print("Detected Emotion:", emotion)

            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(img, emotion.upper(), (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (0, 255, 0), 2, cv2.LINE_AA)

        except Exception as e:
            print("DeepFace Error:", e)

    return img

def process_image(image_path):
    if image_path is None:
        return None

    img = cv2.imread(image_path)
    img = detect_emotion_and_draw(img)
    return img


def process_webcam(img):
    if img is None:
        return None

    img = detect_emotion_and_draw(img)
    return img


def process_video(video_path):
    if video_path is None:
        return None

    cap = cv2.VideoCapture(video_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 20

    output_video = "processed_video.mp4"

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_emotion_and_draw(frame)
        out.write(frame)

    cap.release()
    out.release()

    return output_video

with gr.Blocks() as demo:

    gr.Markdown("# 🂥 Facial Emotion Recognition AI")
    gr.Markdown("Detect emotions from Images, Videos, and Live Webcam")

    with gr.Tab("Image"):
        img_input = gr.Image(type="filepath", label="Upload Image")
        img_output = gr.Image(label="Result")
        btn1 = gr.Button("Detect Emotion")
        btn1.click(process_image, inputs=img_input, outputs=img_output)

    with gr.Tab("Video"):
        vid_input = gr.Video(label="Upload Video")
        vid_output = gr.Video(label="Processed Video")
        btn2 = gr.Button("Detect Emotion")
        btn2.click(process_video, inputs=vid_input, outputs=vid_output)

    with gr.Tab("Webcam"):
        cam_input = gr.Image(sources=["webcam"], type="numpy", label="Open Camera")
        cam_output = gr.Image(label="Result")
        btn3 = gr.Button("Detect Emotion")
        btn3.click(process_webcam, inputs=cam_input, outputs=cam_output)


demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b292b99d50f2fe5094.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
